In [9]:
import sys

# add project root directory to python path to enable import of saapy
if ".." not in sys.path:
    sys.path.append('..')
    
import logging
logging.captureWarnings(True) # mostly warnings caused by self-signed certs

from collections import namedtuple
import itertools
import json
import time
import difflib

# next two liner switchs matplotlib to non-interactive mode
# to stop osx python rocket launcher from jumping in the dock
# ref http://leancrew.com/all-this/2014/01/stopping-the-python-rocketship-icon/
import matplotlib
matplotlib.use("Agg")

import matplotlib.pyplot as plt
# Enable inline plotting
%matplotlib inline

import pandas as pd

from saapy import Workspace, Environment

def setup_env():
    ws = Workspace.from_home("iDempiere", rel_root_dir="Projects", work_dir="iDempiere")
    env = Environment(ws)
    env.setup()
    return env

In [13]:
def export_jira_issues(env):
    issues = list(env.jira.iter_issues('IDEMPIERE'))
    s = json.dumps(issues, indent=4)
    issues_json = env.ws.abs_path('idempiere_jira_issues.json')
    issues_json.write_text(s)

In [ ]:
def build_graph(env):
    export_jira_issues(env)
    export_commit_tree(env)
    commits = lookup_refs_from_commits(env)
    commits[commits['idempiere_jira_issues'].apply(lambda issues: (type(issues) == list and len(issues) > 0))]
    sonar_dups = env.sonar.get("api/duplications/show", dict(
            key="IdempiereKey:temp/org.adempiere.extend/src/test/functional/MBPGroupTest.java"))

In [10]:
def find_reporting_debt():
    query = """
    MATCH 
        (file:JavaFile)-[:DEFINES]->
        (zk_coupled_class:JavaClass)
        -[zk_couple:COUPLES]->
        (zk_class:JavaClass) 
    WHERE 
        zk_class.name =~ {zk_class_name_exp} AND 
        zk_coupled_class.name =~ {class_name_exp}
    WITH file, zk_coupled_class, zk_couple
    MATCH 
        (file:JavaFile)-[:DEFINES]->
        (zk_coupled_class:JavaClass)
        -[db_couple:COUPLES]->
        (db_class:JavaClass) 
    WHERE db_class.name =~ {db_class_name_exp} 
    RETURN DISTINCT file, zk_couple, db_couple
    """
    zk_class_name_exp = "(?i)^org\.zkoss.*"
    class_name_exp = "(?i).*report.*"
    db_class_name_exp = "(?i)^java\.sql.*"
    yield query, dict(zk_class_name_exp=zk_class_name_exp, 
                      db_class_name_exp=db_class_name_exp, 
                      class_name_exp=class_name_exp)
    
def find_class_names():
    query = """
    MATCH (class:JavaClass)
    RETURN class.name as class_name
    """
    yield query, {}

In [14]:
def analyze():
    env = setup_env()
    reporting_debt = env.neo4j.run_in_tx(find_reporting_debt)

In [18]:
def batch_job(l):
    print('batch entered')
    for i in l:
        print('batch loop yielding')
        yield i
        print('batch loop yielded')
    print('batch exiting')

def run_batch(batch, skip=False):
    print(type(batch))
    print('starting batch')
    if not skip:
        for i in batch:
            print('iterating:', i)
        
run_batch(batch_job('abcd'), skip=False)

<class 'generator'>
starting batch
batch entered
batch loop yielding
iterating: a
batch loop yielded
batch loop yielding
iterating: b
batch loop yielded
batch loop yielding
iterating: c
batch loop yielded
batch loop yielding
iterating: d
batch loop yielded
batch exiting


In [25]:
from collections.abc import Generator, Iterable, Iterator

print(isinstance(batch_job([]), Iterable))
print(isinstance(batch_job([]), Iterator))
print(isinstance(batch_job([]), Generator))
print(isinstance([], Iterable))
print(isinstance([], Iterator))
print(isinstance([], Generator))
print(isinstance(iter([]), Iterable))
print(isinstance(iter([]), Iterator))
print(isinstance(iter([]), Generator))

True
True
True
True
False
False
True
True
False


In [29]:
for i in iter([1, 2, 3]): i.next()

AttributeError: 'int' object has no attribute 'next'

In [57]:
from unittest.mock import MagicMock

driver_mock = MagicMock()
session = driver_mock.session()
tx = session.begin_transaction()
with tx:
    tx.run("dummy select", dict(key='value'))
session.close()

<MagicMock name='mock.session().close()' id='4953676096'>

In [76]:
driver_mock.session.return_value.begin_transaction.return_value.run.assert_called_with("dummy select", dict(key='value'))